<a href="https://colab.research.google.com/github/ssd909/first_step_in-ml/blob/master/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
requirements = """
langchain-google-genai
langchain-huggingface
chromadb
sentence-transformers
langchain-community
pypdf
"""


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pypdf

In [ ]:
from pypdf import PdfReader
file_adress = '/content/drive/MyDrive/matsne-1155567-28.pdf'
reader=PdfReader(file_adress)
number_of_pages=len(reader.pages)

In [ ]:
text=""
for page in reader.pages:
  text+= page.extract_text() + "\n"
print(len(text))

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)


In [ ]:
chunks=text_splitter.create_documents([text])
print(len(chunks))

In [ ]:
from google.colab import drive
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma



save_directory = '/content/drive/MyDrive/w_db'


model_name = "paraphrase-multilingual-MiniLM-L12-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=save_directory
)


In [ ]:
import google.generativeai as genai

genai.configure(api_key="")




query = "რომელია საქრთველოს დედაქალაქი?"


embeddings = HuggingFaceEmbeddings(
    model_name="paraphrase-multilingual-MiniLM-L12-v2",

)
db = Chroma(persist_directory="/content/drive/MyDrive/w_db", embedding_function=embeddings)
context =  db.similarity_search(query)

prompt = f"""
გამოიყენე ქვემოთ მოცემული დოკუმენტის ნაწყვეტები კითხვაზე საპასუხოდ.
თუ პასუხი დოკუმენტში არ არის, თქვი, რომ არ იცი.


{context}

 {query}
პასუხი:"""


model = genai.GenerativeModel('gemini-2.5-flash-lite')
response = model.generate_content(prompt)
print(response.text)